In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "24"

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import scanpy as sc

from perturbseq import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
mean_pop = CellPopulation.from_hdf('/data/norman/southark/tfs_standardized/20240311_fibroblast_CRISPRa_mean_pop.hdf', stored_matrix_ids=['p', 'adj_p', 'masked'])
mean_pop.genes.index.name = 'gene_id'

Loading matrix...
Loading metadata...
Done in 2.0715413093566895s.
Done.


In [3]:
guide_library = pd.read_csv('/data/norman/southark/tfs_standardized/20240318_merged_morf_tf_lib_guide_activity_classifications_final_low_UMI_count_regressions_coef.csv', index_col=0)

In [4]:
import dill

with open('/data/norman/southark/tfs_standardized/20240311_fibroblast_CRISPRa_masked_variables.pickle', 'rb') as file:
    data = dill.load(file)

In [5]:
from functools import reduce

all_clustered = {k:v for k,v in data['full_clusters'][1.2].items() if (('non-targeting' not in k) and ('off-target' not in k))}

all_clustered = list(map(lambda x: reduce(np.union1d, list(x.values())), list(all_clustered.values())))
all_clustered = [list(v) for v in all_clustered]
all_clustered = reduce(np.union1d, all_clustered)

In [6]:
new_guides = np.setdiff1d(all_clustered, mean_pop.cells.query('masked_active').index)
new_guides = np.intersect1d(new_guides, guide_library.query('good_guide_fibro').index)
new_guides = mean_pop.cells.loc[new_guides].query('~sequence_driven').index
new_genes = guide_library.loc[new_guides]['target_gene'].unique()
new_genes = np.setdiff1d(new_genes, mean_pop.cells.query('masked_active')['target_gene'])

In [12]:
len(new_genes)

92

In [13]:
mean_pop.cells['expanded_masked_active'] = mean_pop.cells.index.isin(new_guides)

In [24]:
mean_pop.cells['rescued_strong_guide'] = mean_pop.cells.index.isin(new_guides)

In [28]:
table = mean_pop.cells[['expanded_masked_active', 'rescued_strong_guide']]

In [ ]:
table.to_csv('20240319_rescued_fibroblast_guides.csv')